# Reporting on TPC QC for LHC23d1k vs LHC22f apass4

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tracks" data-toc-modified-id="Tracks-1">Tracks</a></span><ul class="toc-item"><li><span><a href="#Phi-Distributions" data-toc-modified-id="Phi-Distributions-1.1">Phi Distributions</a></span></li><li><span><a href="#Eta-distribution" data-toc-modified-id="Eta-distribution-1.2">Eta distribution</a></span></li><li><span><a href="#Transverse-Momentum-(pT)" data-toc-modified-id="Transverse-Momentum-(pT)-1.3">Transverse Momentum (pT)</a></span></li><li><span><a href="#Sign-of-tracks" data-toc-modified-id="Sign-of-tracks-1.4">Sign of tracks</a></span></li><li><span><a href="#Charge-over-transverse-momentum-(q/pT)" data-toc-modified-id="Charge-over-transverse-momentum-(q/pT)-1.5">Charge over transverse momentum (q/pT)</a></span></li><li><span><a href="#Number-of-clusters" data-toc-modified-id="Number-of-clusters-1.6">Number of clusters</a></span></li><li><span><a href="#Number-of-clusters-vs-phi" data-toc-modified-id="Number-of-clusters-vs-phi-1.7">Number of clusters vs phi</a></span></li><li><span><a href="#Number-of-clusters-vs-eta" data-toc-modified-id="Number-of-clusters-vs-eta-1.8">Number of clusters vs eta</a></span></li><li><span><a href="#Number-of-clusters-vs-pT" data-toc-modified-id="Number-of-clusters-vs-pT-1.9">Number of clusters vs pT</a></span></li></ul></li><li><span><a href="#PID" data-toc-modified-id="PID-2">PID</a></span><ul class="toc-item"><li><span><a href="#dEdxTot-of-MIPs" data-toc-modified-id="dEdxTot-of-MIPs-2.1">dEdxTot of MIPs</a></span></li><li><span><a href="#Number-of-clusters-PID" data-toc-modified-id="Number-of-clusters-PID-2.2">Number of clusters PID</a></span></li><li><span><a href="#MIP-dEdxTot-vs-Sector-(vs-Phi-but-both-sides)" data-toc-modified-id="MIP-dEdxTot-vs-Sector-(vs-Phi-but-both-sides)-2.3">MIP dEdxTot vs Sector (vs Phi but both sides)</a></span></li><li><span><a href="#MIP-dEdxTot-vs-Number-of-Clusters" data-toc-modified-id="MIP-dEdxTot-vs-Number-of-Clusters-2.4">MIP dEdxTot vs Number of Clusters</a></span></li><li><span><a href="#MIP-dEdxTot-vs-sin(phi)" data-toc-modified-id="MIP-dEdxTot-vs-sin(phi)-2.5">MIP dEdxTot vs sin(phi)</a></span></li><li><span><a href="#MIP-dEdxTot-vs-tan(lambda)" data-toc-modified-id="MIP-dEdxTot-vs-tan(lambda)-2.6">MIP dEdxTot vs tan(lambda)</a></span></li><li><span><a href="#dEdx-Tot-vs-momentum-(p)" data-toc-modified-id="dEdx-Tot-vs-momentum-(p)-2.7">dEdx Tot vs momentum (p)</a></span></li></ul></li></ul></div>

In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets

from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
from TPCQCVis.src.drawBetheBloch import *

In [ ]:
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
# Read the Root Files for MonteCarlo
path_mc = "/home/berki/cave/alice-tpc-qc/data/sim/2023/LHC23d1k/"
passName_mc = "LHC23d1k"
fileList_mc = glob.glob(path_mc+"*_QC.root")
fileList_mc.sort()
runList_mc = [fileList_mc[i][-14:-8] for i in range(len(fileList_mc))]
rootDataFile_mc = [ROOT.TFile.Open(file,"READ") for file in fileList_mc]
# Back compatibility
runList = runList_mc
rootDataFile = rootDataFile_mc
passName = passName_mc

In [ ]:
# Read the Root Files for data
path_data = "/home/berki/cave/alice-tpc-qc/data/2022/LHC22f/apass4/"
passName_data = "LHC22f"
fileList_data = glob.glob(path_data+"*_QC.root")
fileList_data.sort()
runList_data = [fileList_data[i][-14:-8] for i in range(len(fileList_data))]
rootDataFile_data = [ROOT.TFile.Open(file,"READ") for file in fileList_data]
# Back compatibility
runList_apass1 = runList_data
rootDataFile_apass1 = rootDataFile_data
pass1Name = passName_data

### Tracks

#### Phi Distribution - A Side

In [ ]:
%jsroot on
objectName="hPhiAside"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile_mc,normalize=True,
                                                              legend=False,yAxisRange=[2.0e-3,3.2e-3],legendNames=runList_mc,
                                                              pads=True,compareTo=rootDataFile_data,drawOption="SAME L",
                                                              maxColumns=2,ratio=False,grid=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName_mc, "lep")
leg.AddEntry(histComp[0], passName_data, "lep")

#Sector boundatries
for i in range(len(rootDataFile_mc)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,histogram[i].GetMinimum(),0,histogram[i].GetMaximum())
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,histogram[i].GetMinimum(),(j*2*math.pi)/18,histogram[i].GetMaximum())
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,histogram[i].GetMaximum(),str(j))
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Phi Distribution - C Side

In [ ]:
%jsroot on
objectName="hPhiCside"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile_mc,normalize=True,
                                                              legend=False,yAxisRange=[2.0e-3,3.2e-3],legendNames=runList_mc,
                                                              pads=True,compareTo=rootDataFile_data,drawOption="SAME L",
                                                              maxColumns=2,ratio=False,grid=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName_mc, "lep")
leg.AddEntry(histComp[0], passName_data, "lep")

#Sector boundatries
for i in range(len(rootDataFile_mc)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,histogram[i].GetMinimum(),0,histogram[i].GetMaximum())
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,histogram[i].GetMinimum(),(j*2*math.pi)/18,histogram[i].GetMaximum())
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,histogram[i].GetMaximum(),str(j))
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Eta distribution

In [ ]:
%jsroot on
objectName="hEta"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile_mc,normalize=True,
                                                              legend=False,xAxisRange=[-1.1,1.1],yAxisRange=[2e-3,8e-3],legendNames=runList_mc,
                                                              pads=True,compareTo=rootDataFile_data,drawOption="SAME L",
                                                              maxColumns=2, ratio=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName_mc, "lep")
leg.AddEntry(histComp[0], passName_data, "lep")

#Sector boundatries
for i in range(len(rootDataFile_mc)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Transverse Momentum (pT)

In [ ]:
%jsroot on
objectName="hPt"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile_mc,normalize=True,
                                                              legend=False,yAxisRange=[1e-5,1e-1],legendNames=runList_mc,
                                                              pads=True,compareTo=rootDataFile_data,drawOption="SAME L",
                                                              log="logxy",maxColumns=2,ratio=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName_mc, "lep")
leg.AddEntry(histComp[0], passName_data, "lep")

#Sector boundatries
for i in range(len(rootDataFile_mc)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Sign of tracks

In [ ]:
%jsroot on
objectName="hSign"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,compareTo=rootDataFile_apass1,drawOption="SAME HIST",
                                                              maxColumns=2, ratio=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")
leg.AddEntry(histComp[0], pass1Name, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Charge over transverse momentum (q/pT)

In [ ]:
%jsroot on
objectName="hQOverPt"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,compareTo=rootDataFile_apass1,drawOption="SAME HIST",
                                                              maxColumns=2, ratio=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")
leg.AddEntry(histComp[0], pass1Name, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Number of clusters

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              xAxisRange=[50,160],yAxisRange=[0,42e-3],
                                                              pads=True,compareTo=rootDataFile_apass1,drawOption="SAME HIST",
                                                              maxColumns=2, ratio=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")
leg.AddEntry(histComp[0], pass1Name, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Number of clusters vs phi

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False, yAxisRange = [50,170],legendNames=runList,
                                                              pads=True, drawOption="COLZ", maxColumns=2, ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,50,0,170)
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,50,(j*2*math.pi)/18,170)
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,55,str(j))
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False, yAxisRange = [50,170],legendNames=runList,
                                                              pads=True, drawOption="COLZ", maxColumns=2, ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,50,0,170)
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,50,(j*2*math.pi)/18,170)
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,55,str(j))
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### Number of clusters vs eta

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False, xAxisRange = [-1.1,1.1], yAxisRange = [50,170],legendNames=runList,
                                                              pads=True, drawOption="COLZ", maxColumns=2, ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### Number of clusters vs pT

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False, yAxisRange = [50,170],legendNames=runList,
                                                              log="logx",pads=True, drawOption="COLZ", maxColumns=2,ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

### PID

#### dEdxTot of MIPs

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,compareTo=rootDataFile_apass1,drawOption="SAME HIST",
                                                              log="logx",yAxisRange=[0,0.12],maxColumns=2,ratio=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")
leg.AddEntry(histComp[0], pass1Name, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(1)
        sectorBoundary.SetLineColor(ROOT.kGray+2)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(50,0,50,0.12)
        if typeIndex==0 :
            leg.Draw()

for i,histogram in enumerate(hist+histComp):
    histogram.SetStats(1)
    histogram.Fit("gaus","Sq","",30,70)
    myFit = histogram.GetFunction("gaus")
    print((runList+runList_apass1)[i],": mean=",myFit.GetParameter(1),"sigma=",myFit.GetParameter(2))
    myFit.SetLineColor(histogram.GetLineColor())
    
canvas.cd()
canvas.Draw()

#### Number of clusters PID

In [ ]:
%jsroot on
objectName="hNClsPID"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,compareTo=rootDataFile_apass1,drawOption="SAME HIST",
                                                              xAxisRange=[50,160],yAxisRange=[0,0.045],maxColumns=2,ratio=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")
leg.AddEntry(histComp[0], pass1Name, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### MIP dEdxTot vs Sector (vs Phi but both sides)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False,log="logz",legendNames=runList,
                                                              pads=True, drawOption="COLZ", maxColumns=2,ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(1)
        sectorBoundary.SetLineColor(ROOT.kGray+2)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(0,50,36,50)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### MIP dEdxTot vs Number of Clusters

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False, xAxisRange=[60,160],log="logz",legendNames=runList,
                                                              pads=True, drawOption="COLZ", maxColumns=2, ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(1)
        sectorBoundary.SetLineColor(ROOT.kGray+2)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(60,50,152,50)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### MIP dEdxTot vs sin(phi)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False,log="logz",legendNames=runList,
                                                              pads=True, drawOption="COLZ", maxColumns=2, ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(1)
        sectorBoundary.SetLineColor(ROOT.kGray+2)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(-1,50,1,50)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### MIP dEdxTot vs tan(lambda)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False,log="logz",legendNames=runList,
                                                              pads=True, drawOption="COLZ", maxColumns=2 ,ratio=False)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(1)
        sectorBoundary.SetLineColor(ROOT.kGray+2)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(-2,50,2,50)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### dEdx Tot vs momentum (p)

In [ ]:
%jsroot on
objectName="hdEdxTotVsP_TPC"
[hist,legend,canvas,pad1,histComp,histRatio] = drawHistograms(objectName, rootDataFile, normalize=True, compareTo=rootDataFile_apass1,
                                                              legend=False, legendNames=runList,log="logxyz",
                                                              pads=True, drawOption="COLZ", maxColumns=1)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")
leg2 = ROOT.TLegend()
leg2.AddEntry(histComp[0], pass1Name, "p")
betheBlochLines = [getBetheBloch([run]) for run in runList] 

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist,histComp,histRatio]):
        myPad = padRun.cd(typeIndex+1)
        myPad = drawBetheBloch(betheBlochLines[i],myPad)
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(1)
        sectorBoundary.SetLineColor(ROOT.kGray+2)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(0,50,20,50)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()